<a href="https://colab.research.google.com/github/anwishn/project/blob/main/LG_Aimers_7%EA%B8%B0_1DCNN%2Bbilstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import

In [ ]:
!pip install -r requirements.txt

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [ ]:
import os
import random
import glob
import re

import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler

import torch
import torch.nn as nn
from tqdm import tqdm

from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler

import holidays

import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 설정
LOOKBACK, PREDICT, BATCH_SIZE, EPOCHS = 28, 7, 16, 100
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# SMAPE Loss
def smape_loss(y_pred, y_true):
    denominator = torch.abs(y_true) + torch.abs(y_pred) + 1e-8
    diff = torch.abs(y_pred - y_true)
    loss = 2.0 * diff / denominator
    return loss.mean()

In [ ]:
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

set_seed(42)

# Data Load

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/dacon/open/train/train.csv')

## 파생 변수

In [ ]:
# ...existing code...
def add_features(df):
    import holidays
    kr_holidays = holidays.KR(years=[2023, 2024, 2025])
    df['영업일자'] = pd.to_datetime(df['영업일자'])
    df['요일'] = df['영업일자'].dt.dayofweek
    df['월'] = df['영업일자'].dt.month
    df['일'] = df['영업일자'].dt.day
    df['분기'] = df['영업일자'].dt.quarter
    df['주말여부'] = df['요일'].isin([5, 6]).astype(int)
    df['공휴일여부'] = df['영업일자'].dt.date.isin(kr_holidays).astype(int)
    df['휴일전날'] = df['영업일자'].dt.date.isin([d - pd.Timedelta(days=1) for d in kr_holidays]).astype(int)
    df['휴일다음날'] = df['영업일자'].dt.date.isin([d + pd.Timedelta(days=1) for d in kr_holidays]).astype(int)
    df['월초'] = (df['일'] <= 3).astype(int)
    df['월말'] = (df['일'] >= 28).astype(int)
    df['연도'] = df['영업일자'].dt.year
    df['주차'] = df['영업일자'].dt.isocalendar().week
    df['반기'] = ((df['월'] - 1) // 6 + 1)
    df['월중순'] = ((df['일'] >= 11) & (df['일'] <= 20)).astype(int)
    df['월하순'] = (df['일'] > 20).astype(int)
    df['영업일수'] = df.groupby('영업장명_메뉴명').cumcount() + 1

    # 월/요일 원-핫 인코딩
    for m in range(1, 13):
        df[f'월_{m}'] = (df['월'] == m).astype(int)
    for d in range(0, 7):
        df[f'요일_{d}'] = (df['요일'] == d).astype(int)

    # 시즌/방학/명절 여부
    df['여름방학'] = df['월'].isin([7, 8]).astype(int)
    df['겨울방학'] = df['월'].isin([1, 2]).astype(int)
    df['명절'] = df['월'].isin([1, 2, 9, 10]).astype(int)  # 설, 추석 근접 월

    # 날짜/월/요일 사인/코사인 변환
    df['sin_date'] = np.sin(2 * np.pi * (df['월'] + df['일']/31) / 12)
    df['cos_date'] = np.cos(2 * np.pi * (df['월'] + df['일']/31) / 12)
    df['sin_month'] = np.sin(2 * np.pi * df['월'] / 12)
    df['cos_month'] = np.cos(2 * np.pi * df['월'] / 12)
    df['sin_dayofweek'] = np.sin(2 * np.pi * (df['요일']+1) / 7)
    df['cos_dayofweek'] = np.cos(2 * np.pi * (df['요일']+1) / 7)

    # 7, 14, 21, 27일전 매출 피처 (train/test 모두)
    for lag in [7, 14, 21, 27]:
        if '매출수량' in df.columns:
            df[f'{lag}일전매출'] = df.groupby('영업장명_메뉴명')['매출수량'].shift(lag)
        else:
            df[f'{lag}일전매출'] = np.nan

    # ===== 요일/월별 타겟 통계 피처 =====
    global day_stats, month_stats, month_day_stats

    if 'day_stats' not in globals():
        train_data = df[~df['영업일자'].dt.strftime('%Y-%m-%d').isin(['2024-02-01', '2024-02-02'])]
        day_stats = train_data.groupby(['영업장명_메뉴명', '요일'])['매출수량'].agg(['mean', 'median', 'std']).reset_index()
        day_stats = day_stats.rename(columns={'mean': '요일별_평균', 'median': '요일별_중앙값', 'std': '요일별_표준편차'})
        month_stats = train_data.groupby(['영업장명_메뉴명', '월'])['매출수량'].agg(['mean', 'median', 'std']).reset_index()
        month_stats = month_stats.rename(columns={'mean': '월별_평균', 'median': '월별_중앙값', 'std': '월별_표준편차'})
        month_day_stats = train_data.groupby(['영업장명_메뉴명', '월', '요일'])['매출수량'].agg(['mean']).reset_index()
        month_day_stats = month_day_stats.rename(columns={'mean': '월요일_평균'})

    temp_df = df.merge(day_stats, on=['영업장명_메뉴명', '요일'], how='left')
    temp_df = temp_df.merge(month_stats, on=['영업장명_메뉴명', '월'], how='left')
    temp_df = temp_df.merge(month_day_stats, on=['영업장명_메뉴명', '월', '요일'], how='left')

    for col in ['요일별_평균', '요일별_중앙값', '요일별_표준편차',
                '월별_평균', '월별_중앙값', '월별_표준편차', '월요일_평균']:
        if col in temp_df.columns:
            temp_df[col] = temp_df.groupby('영업장명_메뉴명')[col].transform(
                lambda x: x.fillna(x.mean() if not np.isnan(x.mean()) else 0)
            )
            temp_df[col] = temp_df[col].fillna(0)

    for col in ['요일별_평균', '요일별_중앙값', '요일별_표준편차',
                '월별_평균', '월별_중앙값', '월별_표준편차', '월요일_평균']:
        if col in temp_df.columns:
            df[col] = temp_df[col]

    # 현재 값과 평균의 차이 피처
    if '요일별_평균' in df.columns:
        df['요일평균_대비'] = df['매출수량'] / df['요일별_평균'] - 1
        df['요일평균_대비'] = df['요일평균_대비'].fillna(0).replace([np.inf, -np.inf], 0)

    if '월별_평균' in df.columns:
        df['월평균_대비'] = df['매출수량'] / df['월별_평균'] - 1
        df['월평균_대비'] = df['월평균_대비'].fillna(0).replace([np.inf, -np.inf], 0)

    return df
# ...existing code...

In [ ]:
menu_rfecv_features = {'느티나무 셀프BBQ_1인 수저세트': ['요일',
  '월',
  '일',
  '휴일전날',
  '휴일다음날',
  '연도',
  '주차',
  '월중순',
  '월_2',
  '월_10',
  '요일_1',
  '요일_5',
  'sin_date',
  'cos_date',
  'sin_month',
  'cos_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '느티나무 셀프BBQ_BBQ55(단체)': ['요일', '일', 'sin_date', 'cos_date'],
 '느티나무 셀프BBQ_대여료 30,000원': ['요일', '일', 'sin_date', 'cos_date'],
 '느티나무 셀프BBQ_대여료 60,000원': ['요일',
  '월',
  '일',
  '공휴일여부',
  '휴일전날',
  '휴일다음날',
  '연도',
  '주차',
  '월중순',
  '월_2',
  '월_9',
  '월_11',
  '요일_1',
  '요일_2',
  '요일_4',
  '요일_5',
  'sin_date',
  'cos_date',
  'sin_month',
  'cos_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '느티나무 셀프BBQ_대여료 90,000원': ['요일',
  '월',
  '일',
  '공휴일여부',
  '휴일전날',
  '연도',
  '주차',
  '월중순',
  '월_5',
  '요일_2',
  '요일_4',
  '요일_5',
  'sin_date',
  'cos_date',
  'sin_month',
  'cos_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '느티나무 셀프BBQ_본삼겹 (단품,실내)': ['요일', '일', 'sin_date', 'cos_date'],
 '느티나무 셀프BBQ_스프라이트 (단체)': ['요일', '일', 'sin_date', 'cos_date'],
 '느티나무 셀프BBQ_신라면': ['요일',
  '월',
  '일',
  '공휴일여부',
  '휴일전날',
  '휴일다음날',
  '연도',
  '주차',
  '월중순',
  '월_6',
  '월_10',
  '요일_1',
  '요일_2',
  '요일_3',
  '요일_4',
  '요일_5',
  '여름방학',
  'sin_date',
  'cos_date',
  'sin_month',
  'cos_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '느티나무 셀프BBQ_쌈야채세트': ['요일',
  '일',
  '휴일전날',
  '연도',
  '주차',
  'sin_date',
  'cos_date',
  'sin_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '느티나무 셀프BBQ_쌈장': ['요일',
  '월',
  '일',
  '공휴일여부',
  '휴일전날',
  '연도',
  '주차',
  '월중순',
  '요일_5',
  'sin_date',
  'cos_date',
  'sin_month',
  'cos_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '느티나무 셀프BBQ_육개장 사발면': ['요일',
  '월',
  '일',
  '공휴일여부',
  '연도',
  '주차',
  '월중순',
  '요일_1',
  '요일_5',
  'sin_date',
  'cos_date',
  'sin_dayofweek',
  'cos_dayofweek'],
 '느티나무 셀프BBQ_일회용 소주컵': ['요일', '일', 'sin_date', 'cos_date'],
 '느티나무 셀프BBQ_일회용 종이컵': ['요일', '일', '주차', 'sin_date', 'cos_date'],
 '느티나무 셀프BBQ_잔디그늘집 대여료 (12인석)': ['요일',
  '월',
  '일',
  '공휴일여부',
  '휴일전날',
  '휴일다음날',
  '연도',
  '주차',
  '월중순',
  '요일_2',
  '요일_5',
  'sin_date',
  'cos_date',
  'cos_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '느티나무 셀프BBQ_잔디그늘집 대여료 (6인석)': ['요일',
  '일',
  '연도',
  '주차',
  'sin_date',
  'cos_date',
  'cos_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '느티나무 셀프BBQ_잔디그늘집 의자 추가': ['요일',
  '월',
  '일',
  '공휴일여부',
  '휴일전날',
  '연도',
  '주차',
  '월중순',
  '월_2',
  '월_9',
  '요일_1',
  '요일_4',
  '요일_5',
  'sin_date',
  'cos_date',
  'sin_month',
  'cos_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '느티나무 셀프BBQ_참이슬 (단체)': ['요일', '일', '주차', 'sin_date', 'cos_date'],
 '느티나무 셀프BBQ_친환경 접시 14cm': ['요일', '일', 'sin_date', 'cos_date'],
 '느티나무 셀프BBQ_친환경 접시 23cm': ['요일',
  '월',
  '일',
  '휴일전날',
  '연도',
  '주차',
  '월중순',
  '요일_5',
  'sin_date',
  'cos_date',
  'sin_month',
  'cos_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '느티나무 셀프BBQ_카스 병(단체)': ['요일',
  '월',
  '일',
  '공휴일여부',
  '휴일전날',
  '휴일다음날',
  '연도',
  '주차',
  '월중순',
  '월_6',
  '월_9',
  '월_10',
  '요일_1',
  '요일_2',
  '요일_3',
  '요일_5',
  '여름방학',
  'sin_date',
  'cos_date',
  'sin_month',
  'cos_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '느티나무 셀프BBQ_콜라 (단체)': ['요일',
  '월',
  '일',
  '공휴일여부',
  '휴일전날',
  '휴일다음날',
  '연도',
  '주차',
  '월중순',
  '월_6',
  '요일_1',
  '요일_2',
  '요일_3',
  '요일_4',
  'sin_date',
  'cos_date',
  'cos_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '느티나무 셀프BBQ_햇반': ['요일',
  '월',
  '일',
  '휴일전날',
  '연도',
  '주차',
  '월중순',
  '요일_1',
  'sin_date',
  'cos_date',
  'sin_dayofweek',
  'cos_dayofweek'],
 '느티나무 셀프BBQ_허브솔트': ['요일', '일', 'sin_date', 'cos_date'],
 '담하_(단체) 공깃밥': ['요일', '일', 'sin_date', 'cos_date'],
 '담하_(단체) 생목살 김치전골 2.0': ['요일',
  '월',
  '일',
  '연도',
  '주차',
  '요일_4',
  'sin_date',
  'cos_date',
  'sin_dayofweek',
  'cos_dayofweek'],
 '담하_(단체) 은이버섯 갈비탕': ['일', 'sin_date', 'cos_date'],
 '담하_(단체) 한우 우거지 국밥': ['일', 'sin_date', 'cos_date', 'sin_dayofweek'],
 '담하_(단체) 황태해장국 3/27까지': ['요일',
  '월',
  '일',
  '공휴일여부',
  '휴일전날',
  '휴일다음날',
  '연도',
  '주차',
  '월중순',
  '요일_2',
  '요일_4',
  '요일_5',
  'sin_date',
  'cos_date',
  'sin_month',
  'cos_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '담하_(정식) 된장찌개': ['요일',
  '월',
  '일',
  '공휴일여부',
  '연도',
  '주차',
  '요일_1',
  'sin_date',
  'cos_date',
  'sin_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '담하_(정식) 물냉면 ': ['요일',
  '월',
  '일',
  '공휴일여부',
  '연도',
  '주차',
  '월중순',
  '월_3',
  '요일_1',
  '요일_2',
  'sin_date',
  'cos_date',
  'sin_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '담하_(정식) 비빔냉면': ['요일',
  '월',
  '일',
  '공휴일여부',
  '연도',
  '주차',
  '월중순',
  '요일_1',
  'sin_date',
  'cos_date',
  'cos_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '담하_(후식) 된장찌개': ['요일',
  '월',
  '일',
  '공휴일여부',
  '휴일전날',
  '연도',
  '주차',
  '월중순',
  '월_3',
  '요일_3',
  '요일_5',
  'sin_date',
  'cos_date',
  'sin_month',
  'cos_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '담하_(후식) 물냉면': ['요일',
  '월',
  '일',
  '휴일전날',
  '연도',
  '주차',
  '월중순',
  '요일_5',
  'sin_date',
  'cos_date',
  'sin_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '담하_(후식) 비빔냉면': ['요일',
  '월',
  '일',
  '공휴일여부',
  '휴일전날',
  '연도',
  '주차',
  '월중순',
  '요일_3',
  '요일_5',
  'sin_date',
  'cos_date',
  'sin_month',
  'cos_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '담하_갑오징어 비빔밥': ['요일',
  '월',
  '일',
  '공휴일여부',
  '휴일전날',
  '휴일다음날',
  '연도',
  '주차',
  '월중순',
  '월_2',
  '월_3',
  '월_4',
  '월_5',
  '월_8',
  '월_10',
  '요일_1',
  '요일_2',
  '요일_4',
  '요일_5',
  '여름방학',
  'sin_date',
  'cos_date',
  'cos_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '담하_갱시기': ['일', 'sin_date', 'cos_date'],
 '담하_공깃밥': ['요일',
  '월',
  '일',
  '공휴일여부',
  '휴일전날',
  '연도',
  '주차',
  '월중순',
  '월_3',
  '요일_1',
  '요일_2',
  '요일_3',
  '요일_5',
  'sin_date',
  'cos_date',
  'sin_month',
  'cos_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '담하_꼬막 비빔밥': ['일', 'sin_date', 'cos_date'],
 '담하_느린마을 막걸리': ['요일',
  '월',
  '일',
  '공휴일여부',
  '연도',
  '주차',
  '월중순',
  '요일_1',
  '요일_4',
  '요일_5',
  'sin_date',
  'cos_date',
  'sin_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '담하_담하 한우 불고기': ['요일', '일', 'sin_date', 'cos_date'],
 '담하_담하 한우 불고기 정식': ['요일',
  '월',
  '일',
  '공휴일여부',
  '휴일전날',
  '연도',
  '주차',
  '월중순',
  '월_3',
  '월_4',
  '월_5',
  '월_10',
  '요일_1',
  '요일_2',
  '요일_3',
  '여름방학',
  'sin_date',
  'cos_date',
  'sin_month',
  'cos_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '담하_더덕 한우 지짐': ['요일',
  '월',
  '일',
  '공휴일여부',
  '연도',
  '주차',
  '월중순',
  '요일_5',
  'sin_date',
  'cos_date',
  'cos_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '담하_들깨 양지탕': ['요일',
  '월',
  '일',
  '연도',
  '주차',
  '월중순',
  '월_3',
  '요일_1',
  '요일_5',
  'sin_date',
  'cos_date',
  'cos_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '담하_라면사리': ['일', 'sin_date', 'cos_date'],
 '담하_룸 이용료': ['요일', '일', 'sin_date', 'cos_date'],
 '담하_메밀면 사리': ['요일',
  '월',
  '일',
  '공휴일여부',
  '휴일전날',
  '연도',
  '주차',
  '월중순',
  '월_3',
  '요일_1',
  '요일_2',
  '요일_3',
  '요일_4',
  '요일_5',
  'sin_date',
  'cos_date',
  'sin_month',
  'cos_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '담하_명인안동소주': ['일', 'sin_date', 'cos_date'],
 '담하_명태회 비빔냉면': ['요일',
  '월',
  '일',
  '공휴일여부',
  '휴일전날',
  '연도',
  '주차',
  '월중순',
  '요일_2',
  '요일_5',
  'sin_date',
  'cos_date',
  'sin_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '담하_문막 복분자 칵테일': ['요일',
  '월',
  '일',
  '공휴일여부',
  '연도',
  '주차',
  '월중순',
  '월_5',
  '요일_1',
  '요일_3',
  '요일_4',
  '요일_5',
  'sin_date',
  'cos_date',
  'sin_month',
  'cos_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '담하_봉평메밀 물냉면': ['요일',
  '월',
  '일',
  '휴일전날',
  '연도',
  '주차',
  '월중순',
  '월_3',
  '요일_5',
  'sin_date',
  'cos_date',
  'sin_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '담하_생목살 김치찌개': ['요일',
  '월',
  '일',
  '휴일전날',
  '연도',
  '주차',
  '월_3',
  '요일_1',
  '요일_2',
  'sin_date',
  'cos_date',
  'sin_month',
  'cos_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '담하_스프라이트': ['요일',
  '월',
  '일',
  '공휴일여부',
  '연도',
  '주차',
  '월중순',
  '요일_1',
  '요일_2',
  '요일_3',
  '요일_5',
  'sin_date',
  'cos_date',
  'sin_month',
  'cos_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '담하_은이버섯 갈비탕': ['요일', '일', 'sin_date', 'cos_date'],
 '담하_제로콜라': ['일', 'sin_date', 'cos_date'],
 '담하_참이슬': ['요일', '일', '주차', 'sin_date', 'cos_date'],
 '담하_처음처럼': ['요일',
  '월',
  '일',
  '휴일전날',
  '연도',
  '주차',
  '월중순',
  '월_7',
  '요일_3',
  '요일_5',
  'sin_date',
  'cos_date',
  'sin_dayofweek',
  'cos_dayofweek'],
 '담하_카스': ['요일', '일', 'sin_date', 'cos_date'],
 '담하_콜라': ['요일',
  '월',
  '일',
  '연도',
  '주차',
  '월중순',
  '요일_5',
  'sin_date',
  'cos_date',
  'sin_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '담하_테라': ['요일',
  '월',
  '일',
  '공휴일여부',
  '휴일전날',
  '휴일다음날',
  '연도',
  '주차',
  '월중순',
  '월_2',
  '월_3',
  '월_4',
  '월_5',
  '월_9',
  '요일_1',
  '요일_2',
  '요일_3',
  '요일_4',
  '요일_5',
  '여름방학',
  'sin_date',
  'cos_date',
  'sin_month',
  'cos_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '담하_하동 매실 칵테일': ['요일', '일', '주차', 'sin_date', 'cos_date'],
 '담하_한우 떡갈비 정식': ['요일',
  '월',
  '일',
  '연도',
  '주차',
  'sin_date',
  'cos_date',
  'cos_dayofweek'],
 '담하_한우 미역국 정식': ['요일',
  '월',
  '일',
  '공휴일여부',
  '휴일다음날',
  '연도',
  '주차',
  '월중순',
  '월_3',
  '월_4',
  '월_5',
  '월_7',
  '요일_2',
  '요일_3',
  '요일_4',
  'sin_date',
  'cos_date',
  'sin_month',
  'cos_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '담하_한우 우거지 국밥': ['요일',
  '월',
  '일',
  '공휴일여부',
  '휴일전날',
  '휴일다음날',
  '연도',
  '주차',
  '월중순',
  '월_3',
  '요일_1',
  '요일_2',
  '요일_3',
  '요일_4',
  'sin_date',
  'cos_date',
  'sin_month',
  'cos_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '담하_한우 차돌박이 된장찌개': ['요일',
  '월',
  '일',
  '공휴일여부',
  '연도',
  '주차',
  '월중순',
  '월_3',
  '요일_1',
  '요일_5',
  'sin_date',
  'cos_date',
  'sin_month',
  'cos_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '담하_황태해장국': ['요일',
  '일',
  '공휴일여부',
  '연도',
  '주차',
  '요일_1',
  '요일_3',
  'sin_date',
  'cos_date',
  'sin_month',
  'cos_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '라그로타_AUS (200g)': ['요일',
  '월',
  '일',
  '공휴일여부',
  '휴일전날',
  '휴일다음날',
  '연도',
  '주차',
  '월중순',
  '월_5',
  '요일_1',
  '요일_2',
  '요일_4',
  'sin_date',
  'cos_date',
  'sin_month',
  'cos_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '라그로타_G-Charge(3)': ['요일',
  '월',
  '일',
  '공휴일여부',
  '휴일전날',
  '연도',
  '주차',
  '월_4',
  '요일_2',
  '요일_3',
  '요일_4',
  '요일_5',
  'sin_date',
  'cos_date',
  'cos_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '라그로타_Gls.Sileni': ['요일',
  '월',
  '일',
  '공휴일여부',
  '휴일전날',
  '휴일다음날',
  '연도',
  '주차',
  '월중순',
  '월_2',
  '월_3',
  '월_5',
  '요일_1',
  '요일_2',
  '요일_3',
  '요일_4',
  '요일_5',
  'sin_date',
  'cos_date',
  'sin_month',
  'cos_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '라그로타_Gls.미션 서드': ['요일',
  '월',
  '일',
  '공휴일여부',
  '휴일전날',
  '휴일다음날',
  '연도',
  '주차',
  '월중순',
  '월_3',
  '월_5',
  '요일_1',
  '요일_2',
  '요일_3',
  '요일_4',
  '요일_5',
  'sin_date',
  'cos_date',
  'sin_month',
  'cos_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '라그로타_Open Food': ['요일',
  '월',
  '일',
  '공휴일여부',
  '휴일다음날',
  '연도',
  '주차',
  '월중순',
  '월_3',
  '월_11',
  '요일_1',
  '요일_2',
  '요일_3',
  '요일_4',
  'sin_date',
  'cos_date',
  'cos_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '라그로타_그릴드 비프 샐러드': ['요일',
  '월',
  '일',
  '공휴일여부',
  '휴일전날',
  '휴일다음날',
  '연도',
  '주차',
  '월중순',
  '월_3',
  '월_4',
  '월_5',
  '월_9',
  '요일_1',
  '요일_2',
  '요일_3',
  '요일_4',
  '요일_5',
  '여름방학',
  'sin_date',
  'cos_date',
  'sin_month',
  'cos_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '라그로타_까르보나라': ['요일', '일', '주차', 'sin_date', 'cos_date'],
 '라그로타_모둠 해산물 플래터': ['일', 'sin_date', 'cos_date'],
 '라그로타_미션 서드 카베르네 쉬라': ['요일', '일', 'sin_date', 'cos_date', 'cos_dayofweek'],
 '라그로타_버섯 크림 리조또': ['요일',
  '월',
  '일',
  '공휴일여부',
  '연도',
  '주차',
  '월_5',
  '요일_3',
  '요일_5',
  'sin_date',
  'cos_date',
  'sin_dayofweek',
  'cos_dayofweek'],
 '라그로타_빵 추가 (1인)': ['요일',
  '월',
  '일',
  '공휴일여부',
  '휴일전날',
  '연도',
  '주차',
  '월중순',
  '월_2',
  '월_3',
  '월_8',
  '요일_1',
  '요일_2',
  '요일_3',
  '요일_4',
  '요일_5',
  'sin_date',
  'cos_date',
  'sin_month',
  'cos_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '라그로타_스프라이트': ['요일', '일', '주차', 'sin_date', 'cos_date', 'sin_dayofweek'],
 '라그로타_시저 샐러드 ': ['요일',
  '월',
  '일',
  '공휴일여부',
  '휴일전날',
  '휴일다음날',
  '연도',
  '주차',
  '월중순',
  '월_3',
  '월_4',
  '월_5',
  '월_10',
  '월_11',
  '요일_1',
  '요일_2',
  '요일_3',
  '요일_4',
  '요일_5',
  '여름방학',
  'sin_date',
  'cos_date',
  'sin_month',
  'cos_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '라그로타_아메리카노': ['요일',
  '월',
  '일',
  '공휴일여부',
  '휴일전날',
  '휴일다음날',
  '연도',
  '주차',
  '월중순',
  '월_4',
  '요일_1',
  '요일_2',
  '요일_3',
  '요일_4',
  '요일_5',
  'sin_date',
  'cos_date',
  'sin_month',
  'cos_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '라그로타_알리오 에 올리오 ': ['일', 'sin_date', 'cos_date'],
 '라그로타_양갈비 (4ps)': ['요일',
  '월',
  '일',
  '연도',
  '주차',
  '월중순',
  '요일_2',
  '요일_4',
  '요일_5',
  'sin_date',
  'cos_date',
  'sin_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '라그로타_자몽리치에이드': ['요일',
  '월',
  '일',
  '연도',
  '주차',
  'sin_date',
  'cos_date',
  'sin_month',
  'cos_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '라그로타_제로콜라': ['일', 'sin_date', 'cos_date'],
 '라그로타_카스': ['요일', '일', '주차', 'sin_date', 'cos_date', 'sin_dayofweek'],
 '라그로타_콜라': ['일', 'sin_date', 'cos_date'],
 '라그로타_하이네켄(생)': ['요일',
  '월',
  '일',
  '공휴일여부',
  '휴일전날',
  '휴일다음날',
  '연도',
  '주차',
  '월중순',
  '월_3',
  '요일_2',
  '요일_5',
  'sin_date',
  'cos_date',
  'sin_month',
  'cos_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '라그로타_한우 (200g)': ['요일', '일', '주차', 'sin_date', 'cos_date'],
 '라그로타_해산물 토마토 리조또': ['요일',
  '일',
  '연도',
  '주차',
  'sin_date',
  'cos_date',
  'sin_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '라그로타_해산물 토마토 스튜 파스타': ['요일',
  '월',
  '일',
  '연도',
  '주차',
  '요일_2',
  '요일_3',
  'sin_date',
  'cos_date',
  'cos_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '라그로타_해산물 토마토 스파게티': ['요일',
  '일',
  '휴일전날',
  '연도',
  '주차',
  '월_3',
  'sin_date',
  'cos_date',
  'cos_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '미라시아_(단체)브런치주중 36,000': ['요일',
  '월',
  '일',
  '휴일다음날',
  '연도',
  '주차',
  '월중순',
  '요일_2',
  'sin_date',
  'cos_date',
  'sin_month',
  'cos_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '미라시아_(오븐) 하와이안 쉬림프 피자': ['요일',
  '월',
  '일',
  '공휴일여부',
  '휴일전날',
  '휴일다음날',
  '연도',
  '주차',
  '월중순',
  '월_2',
  '월_4',
  '월_5',
  '월_10',
  '요일_1',
  '요일_2',
  '요일_3',
  '요일_4',
  '요일_5',
  'sin_date',
  'cos_date',
  'sin_month',
  'cos_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '미라시아_(화덕) 불고기 페퍼로니 반반피자': ['요일',
  '월',
  '일',
  '공휴일여부',
  '휴일전날',
  '휴일다음날',
  '연도',
  '주차',
  '월중순',
  '월_3',
  '월_5',
  '월_6',
  '월_10',
  '요일_1',
  '요일_2',
  '요일_3',
  '요일_4',
  '요일_5',
  '여름방학',
  'sin_date',
  'cos_date',
  'sin_month',
  'cos_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '미라시아_BBQ Platter': ['요일',
  '월',
  '일',
  '공휴일여부',
  '휴일전날',
  '연도',
  '주차',
  '월중순',
  '월_2',
  '월_3',
  '월_4',
  '월_5',
  '요일_2',
  '요일_3',
  '요일_4',
  '요일_5',
  'sin_date',
  'cos_date',
  'sin_month',
  'cos_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '미라시아_BBQ 고기추가': ['요일', '일', '주차', 'sin_date', 'cos_date'],
 '미라시아_공깃밥': ['요일',
  '월',
  '일',
  '연도',
  '주차',
  '월중순',
  '요일_5',
  'sin_date',
  'cos_date',
  'sin_dayofweek',
  'cos_dayofweek'],
 '미라시아_글라스와인 (레드)': ['일', 'sin_date', 'cos_date'],
 '미라시아_레인보우칵테일(알코올)': ['일', 'sin_date', 'cos_date'],
 '미라시아_미라시아 브런치 (패키지)': ['요일',
  '일',
  '주차',
  'sin_date',
  'cos_date',
  'sin_dayofweek'],
 '미라시아_버드와이저(무제한)': ['요일', '일', 'sin_date', 'cos_date'],
 '미라시아_보일링 랍스타 플래터': ['요일',
  '월',
  '일',
  '휴일전날',
  '휴일다음날',
  '연도',
  '주차',
  'sin_date',
  'cos_date',
  'sin_dayofweek',
  'cos_dayofweek'],
 '미라시아_보일링 랍스타 플래터(덜매운맛)': ['요일',
  '월',
  '일',
  '휴일전날',
  '휴일다음날',
  '연도',
  '주차',
  '월중순',
  '월_2',
  '월_9',
  '요일_2',
  '요일_3',
  '요일_5',
  'sin_date',
  'cos_date',
  'sin_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '미라시아_브런치 2인 패키지 ': ['요일', '일', '주차', 'sin_date', 'cos_date'],
 '미라시아_브런치 4인 패키지 ': ['요일',
  '월',
  '일',
  '휴일전날',
  '연도',
  '주차',
  '월중순',
  '요일_3',
  '요일_4',
  '요일_5',
  'sin_date',
  'cos_date',
  'sin_month',
  'cos_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '미라시아_브런치(대인) 주말': ['요일',
  '월',
  '일',
  '공휴일여부',
  '연도',
  '주차',
  '월중순',
  '월_3',
  'sin_date',
  'cos_date',
  'sin_month',
  'cos_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '미라시아_브런치(대인) 주중': ['요일',
  '월',
  '일',
  '공휴일여부',
  '휴일다음날',
  '연도',
  '주차',
  '월중순',
  '월_3',
  '요일_1',
  '요일_3',
  '요일_5',
  'sin_date',
  'cos_date',
  'sin_month',
  'cos_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '미라시아_브런치(어린이)': ['요일', '일', '주차', 'sin_date', 'cos_date'],
 '미라시아_쉬림프 투움바 파스타': ['요일',
  '월',
  '일',
  '휴일전날',
  '연도',
  '주차',
  '월중순',
  '요일_5',
  'sin_date',
  'cos_date',
  'sin_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '미라시아_스텔라(무제한)': ['요일',
  '월',
  '일',
  '공휴일여부',
  '휴일전날',
  '휴일다음날',
  '연도',
  '주차',
  '월중순',
  '월_3',
  '요일_1',
  '요일_5',
  'sin_date',
  'cos_date',
  'sin_month',
  'cos_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '미라시아_스프라이트': ['요일',
  '월',
  '일',
  '공휴일여부',
  '휴일전날',
  '연도',
  '주차',
  '월중순',
  '요일_4',
  '요일_5',
  'sin_date',
  'cos_date',
  'sin_month',
  'cos_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '미라시아_애플망고 에이드': ['요일',
  '월',
  '일',
  '공휴일여부',
  '휴일전날',
  '휴일다음날',
  '연도',
  '주차',
  '월중순',
  '월_5',
  '요일_5',
  'sin_date',
  'cos_date',
  'cos_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '미라시아_얼그레이 하이볼': ['요일', '일', 'sin_date', 'cos_date'],
 '미라시아_오븐구이 윙과 킬바사소세지': ['요일',
  '월',
  '일',
  '공휴일여부',
  '휴일전날',
  '연도',
  '주차',
  '월중순',
  '월_3',
  '월_4',
  '월_10',
  '요일_1',
  '요일_2',
  '요일_5',
  'sin_date',
  'cos_date',
  'cos_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '미라시아_유자 하이볼': ['요일',
  '월',
  '일',
  '휴일전날',
  '연도',
  '주차',
  '월중순',
  '요일_5',
  'sin_date',
  'cos_date',
  'sin_month',
  'cos_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '미라시아_잭 애플 토닉': ['일', 'sin_date', 'cos_date'],
 '미라시아_칠리 치즈 프라이': ['요일',
  '월',
  '일',
  '휴일전날',
  '연도',
  '주차',
  '월중순',
  '요일_5',
  'sin_date',
  'cos_date',
  'sin_dayofweek',
  'cos_dayofweek'],
 '미라시아_코카콜라': ['요일',
  '일',
  '연도',
  '주차',
  'sin_date',
  'cos_date',
  'sin_dayofweek',
  'cos_dayofweek'],
 '미라시아_코카콜라(제로)': ['요일',
  '월',
  '일',
  '공휴일여부',
  '휴일전날',
  '연도',
  '주차',
  '월중순',
  '월_3',
  '월_5',
  '요일_1',
  '요일_2',
  '요일_3',
  '요일_4',
  '요일_5',
  'sin_date',
  'cos_date',
  'sin_month',
  'cos_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '미라시아_콥 샐러드': ['요일', '일', '주차', 'sin_date', 'cos_date'],
 '미라시아_파스타면 추가(150g)': ['요일',
  '월',
  '일',
  '공휴일여부',
  '휴일전날',
  '연도',
  '주차',
  '월중순',
  '요일_2',
  '요일_5',
  'sin_date',
  'cos_date',
  'sin_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '미라시아_핑크레몬에이드': ['요일',
  '월',
  '일',
  '공휴일여부',
  '휴일전날',
  '연도',
  '주차',
  '월중순',
  '요일_2',
  '요일_3',
  '요일_4',
  '요일_5',
  'sin_date',
  'cos_date',
  'sin_month',
  'cos_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '연회장_Cass Beer': ['요일',
  '월',
  '일',
  '연도',
  '주차',
  '월중순',
  '월_4',
  '요일_2',
  '요일_3',
  '요일_4',
  'sin_date',
  'cos_date',
  'sin_month',
  'cos_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '연회장_Conference L1': ['요일',
  '일',
  '연도',
  '주차',
  'sin_date',
  'cos_date',
  'sin_dayofweek',
  'cos_dayofweek'],
 '연회장_Conference L2': ['요일', '일', '연도', '주차', 'sin_date', 'cos_date'],
 '연회장_Conference L3': ['요일',
  '월',
  '일',
  '공휴일여부',
  '휴일전날',
  '휴일다음날',
  '연도',
  '주차',
  '월중순',
  '월_2',
  '월_3',
  '월_4',
  '월_5',
  '월_6',
  '요일_1',
  '요일_2',
  '요일_3',
  '요일_4',
  '요일_5',
  'sin_date',
  'cos_date',
  'sin_month',
  'cos_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '연회장_Conference M1': ['요일',
  '월',
  '일',
  '공휴일여부',
  '휴일전날',
  '휴일다음날',
  '연도',
  '주차',
  '월중순',
  '월_3',
  '월_5',
  '요일_1',
  '요일_2',
  '요일_3',
  '요일_4',
  '여름방학',
  'sin_date',
  'cos_date',
  'sin_month',
  'cos_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '연회장_Conference M8': ['요일',
  '월',
  '일',
  '공휴일여부',
  '연도',
  '주차',
  '요일_1',
  '요일_2',
  '요일_3',
  '요일_4',
  'sin_date',
  'cos_date',
  'sin_month',
  'cos_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '연회장_Conference M9': ['요일', '일', 'sin_date', 'cos_date'],
 '연회장_Convention Hall': ['요일',
  '월',
  '일',
  '휴일전날',
  '연도',
  '주차',
  '월중순',
  '요일_1',
  '요일_3',
  '요일_4',
  'sin_date',
  'cos_date',
  'sin_month',
  'cos_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '연회장_Cookie Platter': ['요일',
  '월',
  '일',
  '공휴일여부',
  '휴일전날',
  '연도',
  '주차',
  '월중순',
  '월_2',
  '월_4',
  '요일_1',
  '요일_2',
  '요일_3',
  '요일_4',
  'sin_date',
  'cos_date',
  'sin_month',
  'cos_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '연회장_Grand Ballroom': ['요일', '일', '주차', 'sin_date', 'cos_date'],
 '연회장_OPUS 2': ['요일',
  '월',
  '일',
  '연도',
  '주차',
  '월중순',
  '요일_3',
  '요일_4',
  'sin_date',
  'cos_date',
  'sin_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '연회장_Regular Coffee': ['요일',
  '월',
  '일',
  '공휴일여부',
  '휴일전날',
  '연도',
  '주차',
  '월중순',
  '월_4',
  '월_9',
  '요일_1',
  '요일_2',
  '요일_3',
  '요일_4',
  'sin_date',
  'cos_date',
  'sin_month',
  'cos_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '연회장_골뱅이무침': ['일', 'sin_date', 'cos_date'],
 '연회장_공깃밥': ['요일',
  '월',
  '일',
  '공휴일여부',
  '휴일전날',
  '휴일다음날',
  '연도',
  '주차',
  '월중순',
  '월_2',
  '월_3',
  '요일_1',
  '요일_2',
  '요일_3',
  '요일_4',
  '요일_5',
  'sin_date',
  'cos_date',
  'sin_month',
  'cos_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '연회장_돈목살 김치찌개 (밥포함)': ['일', 'sin_date', 'cos_date'],
 '연회장_로제 치즈떡볶이': ['일', 'sin_date', 'cos_date'],
 '연회장_마라샹궈': ['일', 'sin_date', 'cos_date', 'sin_dayofweek'],
 '연회장_매콤 무뼈닭발&계란찜': ['일', 'sin_date', 'cos_date'],
 '연회장_모둠 돈육구이(3인)': ['일', 'sin_date', 'cos_date'],
 '연회장_삼겹살추가 (200g)': ['요일',
  '일',
  '연도',
  '주차',
  'sin_date',
  'cos_date',
  'sin_dayofweek'],
 '연회장_야채추가': ['요일', '일', '주차', 'sin_date', 'cos_date'],
 '연회장_왕갈비치킨': ['요일',
  '월',
  '일',
  '공휴일여부',
  '휴일전날',
  '연도',
  '주차',
  '월중순',
  '요일_5',
  'sin_date',
  'cos_date',
  'sin_month',
  'cos_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '연회장_주먹밥 (2ea)': ['일', 'sin_date', 'cos_date'],
 '카페테리아_공깃밥(추가)': ['요일', '일', '주차', 'sin_date', 'cos_date'],
 '카페테리아_구슬아이스크림': ['일', 'sin_date', 'cos_date'],
 '카페테리아_단체식 13000(신)': ['일', 'sin_date', 'cos_date', 'sin_dayofweek'],
 '카페테리아_단체식 18000(신)': ['요일',
  '월',
  '일',
  '공휴일여부',
  '연도',
  '주차',
  '월중순',
  '요일_2',
  '요일_3',
  '요일_4',
  'sin_date',
  'cos_date',
  'sin_month',
  'cos_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '카페테리아_돼지고기 김치찌개': ['요일',
  '월',
  '일',
  '공휴일여부',
  '휴일전날',
  '연도',
  '주차',
  '월중순',
  '월_4',
  '요일_2',
  '요일_3',
  '요일_4',
  '요일_5',
  'sin_date',
  'cos_date',
  'sin_month',
  'cos_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '카페테리아_복숭아 아이스티': ['일', 'sin_date', 'cos_date'],
 '카페테리아_새우 볶음밥': ['요일', '일', 'sin_date', 'cos_date'],
 '카페테리아_새우튀김 우동': ['일', 'sin_date', 'cos_date'],
 '카페테리아_샷 추가': ['일', 'sin_date', 'cos_date'],
 '카페테리아_수제 등심 돈까스': ['요일', '일', '주차', 'sin_date', 'cos_date'],
 '카페테리아_아메리카노(HOT)': ['일', 'sin_date', 'cos_date'],
 '카페테리아_아메리카노(ICE)': ['일', 'sin_date', 'cos_date'],
 '카페테리아_약 고추장 돌솥비빔밥': ['요일', '일', '연도', '주차', 'sin_date', 'cos_date'],
 '카페테리아_어린이 돈까스': ['요일', '일', 'sin_date', 'cos_date'],
 '카페테리아_오픈푸드': ['일', 'sin_date', 'cos_date'],
 '카페테리아_진사골 설렁탕': ['일', 'sin_date', 'cos_date'],
 '카페테리아_짜장면': ['요일',
  '월',
  '일',
  '연도',
  '주차',
  '월중순',
  'sin_date',
  'cos_date',
  'sin_dayofweek',
  'cos_dayofweek'],
 '카페테리아_짜장밥': ['요일',
  '일',
  '연도',
  '주차',
  'sin_date',
  'cos_date',
  'sin_dayofweek',
  'cos_dayofweek'],
 '카페테리아_짬뽕': ['요일', '일', 'sin_date', 'cos_date'],
 '카페테리아_짬뽕밥': ['요일',
  '월',
  '일',
  '공휴일여부',
  '휴일전날',
  '연도',
  '주차',
  '월중순',
  '요일_1',
  '요일_2',
  '요일_4',
  '요일_5',
  'sin_date',
  'cos_date',
  'sin_month',
  'cos_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '카페테리아_치즈돈까스': ['요일',
  '월',
  '일',
  '공휴일여부',
  '휴일전날',
  '연도',
  '주차',
  '월중순',
  '월_4',
  '요일_2',
  '요일_5',
  '여름방학',
  'sin_date',
  'cos_date',
  'sin_month',
  'cos_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '카페테리아_카페라떼(HOT)': ['일', 'sin_date', 'cos_date'],
 '카페테리아_카페라떼(ICE)': ['일', 'sin_date', 'cos_date'],
 '카페테리아_한상 삼겹구이 정식(2인) 소요시간 약 15~20분': ['요일',
  '월',
  '일',
  '공휴일여부',
  '휴일전날',
  '휴일다음날',
  '연도',
  '주차',
  '월중순',
  '월_6',
  '요일_1',
  '요일_2',
  '요일_5',
  'sin_date',
  'cos_date',
  'sin_month',
  'cos_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '포레스트릿_꼬치어묵': ['요일',
  '월',
  '일',
  '공휴일여부',
  '휴일다음날',
  '연도',
  '주차',
  '월중순',
  '요일_2',
  'sin_date',
  'cos_date',
  'cos_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '포레스트릿_떡볶이': ['요일', '일', '주차', 'sin_date', 'cos_date'],
 '포레스트릿_복숭아 아이스티': ['요일',
  '월',
  '일',
  '공휴일여부',
  '연도',
  '주차',
  'sin_date',
  'cos_date',
  'sin_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '포레스트릿_생수': ['요일',
  '월',
  '일',
  '연도',
  '주차',
  'sin_date',
  'cos_date',
  'sin_dayofweek',
  'cos_dayofweek'],
 '포레스트릿_스프라이트': ['요일',
  '월',
  '일',
  '연도',
  '주차',
  'sin_date',
  'cos_date',
  'sin_dayofweek'],
 '포레스트릿_아메리카노(HOT)': ['요일',
  '월',
  '일',
  '공휴일여부',
  '연도',
  '주차',
  '월중순',
  '월_4',
  '요일_2',
  'sin_date',
  'cos_date',
  'cos_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '포레스트릿_아메리카노(ICE)': ['요일',
  '월',
  '일',
  '공휴일여부',
  '연도',
  '주차',
  'sin_date',
  'cos_date',
  'sin_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '포레스트릿_치즈 핫도그': ['요일',
  '월',
  '일',
  '공휴일여부',
  '연도',
  '주차',
  '월중순',
  '월_4',
  '월_5',
  '요일_2',
  '요일_5',
  'sin_date',
  'cos_date',
  'sin_month',
  'cos_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '포레스트릿_카페라떼(HOT)': ['요일',
  '월',
  '일',
  '공휴일여부',
  '휴일다음날',
  '연도',
  '주차',
  '월중순',
  '월_4',
  '월_5',
  '월_10',
  '요일_1',
  '요일_5',
  'sin_date',
  'cos_date',
  'cos_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '포레스트릿_카페라떼(ICE)': ['요일',
  '월',
  '일',
  '공휴일여부',
  '휴일전날',
  '휴일다음날',
  '연도',
  '주차',
  '월중순',
  '월_3',
  '월_4',
  '월_5',
  '월_10',
  '요일_3',
  'sin_date',
  'cos_date',
  'sin_month',
  'cos_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '포레스트릿_코카콜라': ['요일',
  '월',
  '일',
  '공휴일여부',
  '휴일전날',
  '휴일다음날',
  '연도',
  '주차',
  '월중순',
  '월_5',
  '요일_3',
  '요일_5',
  'sin_date',
  'cos_date',
  'sin_month',
  'cos_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '포레스트릿_페스츄리 소시지': ['요일',
  '월',
  '일',
  '공휴일여부',
  '휴일전날',
  '연도',
  '주차',
  '월중순',
  '월_4',
  '요일_5',
  'sin_date',
  'cos_date',
  'sin_month',
  'cos_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '화담숲주막_느린마을 막걸리': ['요일',
  '월',
  '일',
  '공휴일여부',
  '휴일전날',
  '연도',
  '주차',
  '월중순',
  '월_2',
  '월_4',
  '월_5',
  '월_10',
  '요일_2',
  '요일_4',
  '요일_5',
  '여름방학',
  'sin_date',
  'cos_date',
  'sin_month',
  'cos_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '화담숲주막_단호박 식혜 ': ['요일', '일', '주차', 'sin_date', 'cos_date'],
 '화담숲주막_병천순대': ['요일', '일', '주차', 'sin_date', 'cos_date'],
 '화담숲주막_스프라이트': ['요일',
  '월',
  '일',
  '공휴일여부',
  '연도',
  '주차',
  '월_4',
  'sin_date',
  'cos_date',
  'sin_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '화담숲주막_참살이 막걸리': ['요일',
  '월',
  '일',
  '공휴일여부',
  '연도',
  '주차',
  '월중순',
  '여름방학',
  'sin_date',
  'cos_date',
  'sin_month',
  'cos_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '화담숲주막_찹쌀식혜': ['요일', '일', '주차', 'sin_date', 'cos_date'],
 '화담숲주막_콜라': ['요일',
  '월',
  '일',
  '공휴일여부',
  '휴일전날',
  '연도',
  '주차',
  'sin_date',
  'cos_date',
  'sin_month',
  'cos_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '화담숲주막_해물파전': ['요일',
  '월',
  '일',
  '공휴일여부',
  '휴일전날',
  '연도',
  '주차',
  'sin_date',
  'cos_date',
  'cos_month',
  'cos_dayofweek'],
 '화담숲카페_메밀미숫가루': ['요일',
  '월',
  '일',
  '휴일전날',
  '연도',
  '주차',
  '월중순',
  'sin_date',
  'cos_date',
  'cos_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '화담숲카페_아메리카노 HOT': ['요일',
  '월',
  '일',
  '공휴일여부',
  '휴일전날',
  '연도',
  '주차',
  '월중순',
  '월_3',
  '월_4',
  '월_5',
  '요일_3',
  '요일_4',
  '요일_5',
  '여름방학',
  'sin_date',
  'cos_date',
  'sin_month',
  'cos_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '화담숲카페_아메리카노 ICE': ['요일',
  '월',
  '일',
  '공휴일여부',
  '휴일전날',
  '휴일다음날',
  '연도',
  '주차',
  '월중순',
  '월_5',
  '월_10',
  '요일_2',
  '요일_3',
  '요일_4',
  '요일_5',
  '여름방학',
  'sin_date',
  'cos_date',
  'sin_month',
  'cos_month',
  'sin_dayofweek',
  'cos_dayofweek'],
 '화담숲카페_카페라떼 ICE': ['요일',
  '일',
  '주차',
  'sin_date',
  'cos_date',
  'sin_dayofweek',
  'cos_dayofweek'],
 '화담숲카페_현미뻥스크림': ['요일',
  '월',
  '일',
  '공휴일여부',
  '연도',
  '주차',
  '월중순',
  '요일_4',
  '여름방학',
  'sin_date',
  'cos_date',
  'sin_month',
  'sin_dayofweek',
  'cos_dayofweek']}

# Modeling

## 검증 코드

In [ ]:
import numpy as np
import pandas as pd
import torch
import ast

# --- 키 정규화: tuple/문자열 튜플/콤마/괄호 등 통일 ---
def normalize_menu_key(k):
    if isinstance(k, tuple):
        k = k[0]
    s = str(k)
    if s.startswith("(") and s.endswith(")"):
        try:
            t = ast.literal_eval(s)
            if isinstance(t, (tuple, list)) and len(t) > 0:
                s = t[0]
        except Exception:
            s = s.strip("()")
            if s.endswith(","):
                s = s[:-1]
    return str(s).strip()

# --- SMAPE (원스케일, 정답 0 마스킹) ---
def smape_np(y_true, y_pred, eps: float = 1e-6):
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)
    finite = np.isfinite(y_true) & np.isfinite(y_pred)
    y_true, y_pred = y_true[finite], y_pred[finite]
    mask = y_true != 0
    if not np.any(mask):
        return np.nan
    y_true, y_pred = y_true[mask], y_pred[mask]
    denom = np.abs(y_true) + np.abs(y_pred) + eps
    return float(np.mean(2.0 * np.abs(y_pred - y_true) / denom) * 100.0)

# --- 롤링 검증 (날짜는 이미 실날짜, 로그/exp 없음) ---
def rolling_validate_real_dates(trained_models, train_df, test_df):
    # 0) 전처리 + 키 정규화
    train_df = add_features(train_df.copy())
    test_df  = add_features(test_df.copy())
    for df_ in (train_df, test_df):
        df_.loc[df_['매출수량'] < 0, '매출수량'] = 0
        df_['영업일자'] = pd.to_datetime(df_['영업일자']).dt.normalize()
        df_['영업장명_메뉴명_norm'] = df_['영업장명_메뉴명'].apply(normalize_menu_key)

    # 학습 모델 키도 정규화
    tm = {normalize_menu_key(k): v for k, v in trained_models.items()}

    # 테스트에 등장하는 키만 평가
    keys_in_test = set(test_df['영업장명_메뉴명_norm'].astype(str).unique())

    # train+test 합쳐 연속 시계열 확보
    full_df = pd.concat([train_df, test_df], ignore_index=True)
    full_df = full_df.sort_values(['영업장명_메뉴명_norm', '영업일자'])

    rows_series, rows_window, long_rows = [], [], []
    all_true_concat, all_pred_concat = [], []

    # 테스트 구간의 날짜 범위(키별로 다를 수 있으니 키별로 뽑음)
    for key_norm, g in full_df.groupby('영업장명_메뉴명_norm'):
        key_str = str(key_norm)
        if (key_str not in tm) or (key_str not in keys_in_test):
            continue

        pack = tm[key_str]
        model = pack['model']
        scaler = pack['scaler']
        feat_cols = pack['feature_cols']

        g = g.sort_values('영업일자')
        dates = pd.to_datetime(g['영업일자']).dt.normalize()
        date_to_idx = pd.Series(range(len(dates)), index=dates).to_dict()

        # 이 키의 test 기간(실제 존재하는 날짜 범위)
        g_test = test_df.loc[test_df['영업장명_메뉴명_norm'] == key_str].sort_values('영업일자')
        if g_test.empty:
            continue
        test_start = g_test['영업일자'].min()
        test_end   = g_test['영업일자'].max()

        # 원스케일 정답, 스케일 입력
        y_all = g['매출수량'].to_numpy()
        X_all = g[feat_cols].to_numpy()
        X_scaled = scaler.transform(X_all)

        cur_start = test_start
        last_start = test_end - pd.Timedelta(days=PREDICT-1)

        series_preds_concat, series_trues_concat = [], []
        n_windows_ok = 0

        while cur_start <= last_start:
            win_end_dt   = cur_start - pd.Timedelta(days=1)
            win_start_dt = win_end_dt - pd.Timedelta(days=LOOKBACK-1)

            # 입력 인덱스 수집
            have_in, in_idxs = True, []
            for k in range(LOOKBACK):
                d = win_start_dt + pd.Timedelta(days=k)
                idx = date_to_idx.get(d, None)
                if idx is None: have_in = False; break
                in_idxs.append(idx)

            # 출력 인덱스 수집
            have_out, out_idxs = True, []
            for k in range(PREDICT):
                d = cur_start + pd.Timedelta(days=k)
                idx = date_to_idx.get(d, None)
                if idx is None: have_out = False; break
                out_idxs.append(idx)

            if have_in and have_out:
                x_win = X_scaled[in_idxs]          # (LOOKBACK, F)
                y_true_win = y_all[out_idxs]       # (PREDICT,)

                # 전부 0이면 스킵
                if np.all(y_true_win == 0):
                    cur_start += pd.Timedelta(days=1)
                    continue

                # 예측(스케일) → 역스케일(매출수량만)
                x_t = torch.tensor(x_win, dtype=torch.float32, device=DEVICE).unsqueeze(0)
                with torch.no_grad():
                    pred_scaled = model(x_t).squeeze(0).cpu().numpy()  # (PREDICT,)

                dummy = np.zeros((PREDICT, len(feat_cols)), dtype=float)
                dummy[:, 0] = pred_scaled
                y_pred = scaler.inverse_transform(dummy)[:, 0]
                y_pred = np.maximum(y_pred, 0.0)

                s_win = smape_np(y_true_win, y_pred)
                if np.isfinite(s_win):
                    rows_window.append({
                        '영업장명_메뉴명': key_str,
                        'window_start': win_start_dt.strftime('%Y-%m-%d'),
                        'window_end':   win_end_dt.strftime('%Y-%m-%d'),
                        'horizon_start': cur_start.strftime('%Y-%m-%d'),
                        'horizon_end':   (cur_start + pd.Timedelta(days=PREDICT-1)).strftime('%Y-%m-%d'),
                        'n_points': PREDICT,
                        'SMAPE(%)': s_win
                    })
                    for d_i, d in enumerate(pd.date_range(cur_start, periods=PREDICT, freq='D')):
                        long_rows.append({
                            '영업장명_메뉴명': key_str,
                            '영업일자': d.strftime('%Y-%m-%d'),
                            '예측': float(y_pred[d_i]),
                            '정답': float(y_true_win[d_i]),
                            'window_start': win_start_dt.strftime('%Y-%m-%d')
                        })
                    series_preds_concat.append(y_pred)
                    series_trues_concat.append(y_true_win)
                    n_windows_ok += 1

            cur_start += pd.Timedelta(days=1)

        if n_windows_ok > 0:
            yp = np.concatenate(series_preds_concat, axis=0)
            yt = np.concatenate(series_trues_concat, axis=0)
            s_series = smape_np(yt, yp)
            if np.isfinite(s_series):
                rows_series.append({
                    '영업장명_메뉴명': key_str,
                    'n_windows': n_windows_ok,
                    'n_points': len(yp),
                    'SMAPE(%)': s_series
                })
                all_pred_concat.append(yp)
                all_true_concat.append(yt)

    per_series_df = (
        pd.DataFrame(rows_series).sort_values('SMAPE(%)')
        if rows_series else
        pd.DataFrame(columns=['영업장명_메뉴명','n_windows','n_points','SMAPE(%)'])
    )
    per_window_df = (
        pd.DataFrame(rows_window).sort_values(['영업장명_메뉴명','window_start'])
        if rows_window else
        pd.DataFrame(columns=['영업장명_메뉴명','window_start','window_end','horizon_start','horizon_end','n_points','SMAPE(%)'])
    )
    long_df = (
        pd.DataFrame(long_rows)
        if long_rows else
        pd.DataFrame(columns=['영업장명_메뉴명','영업일자','예측','정답','window_start'])
    )
    overall = smape_np(np.concatenate(all_true_concat), np.concatenate(all_pred_concat)) if all_true_concat else np.nan
    return overall, per_series_df, per_window_df, long_df


# 메뉴별 193개 평균 SMAPE 테이블 생성
def menu_smape_table(long_df):
    rows = []
    for menu, g in long_df.groupby('영업장명_메뉴명'):
        y_true = g['정답'].to_numpy()
        y_pred = g['예측'].to_numpy()
        score = smape_np(y_true, y_pred)
        rows.append({'영업장명_메뉴명': menu, 'n_days': len(y_true), 'SMAPE(%)': score})
    df = pd.DataFrame(rows).sort_values('SMAPE(%)')
    return df

In [ ]:
# TEST_00로 검증 예측
def predict_next_28_from_train(trained_models, keys=None):
    preds = {}
    for key, pack in trained_models.items():
        key_str = key[0] if isinstance(key, tuple) else key  # tuple → 문자열 변환
        if (keys is not None) and (key_str not in keys):
            continue
        model = pack['model']
        scaler = pack['scaler']
        feature_cols = pack['feature_cols']
        recent_scaled = pack['last_sequence']  # 이미 scaled

        x_input = torch.tensor([recent_scaled]).float().to(DEVICE)
        with torch.no_grad():
            pred_scaled = model(x_input).squeeze().cpu().numpy()

        # 역스케일 (매출수량만 역변환)
        restored = []
        for v in pred_scaled:
            dummy = np.zeros((1, len(feature_cols)))
            dummy[0, 0] = v
            inv = scaler.inverse_transform(dummy)[0, 0]
            restored.append(max(inv, 0.0))
        preds[key_str] = np.array(restored, dtype=float)
    return preds

# TEST_00 정답과 SMAPE 평가
def smape_np(y_true, y_pred, eps: float = 1e-6):
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    mask = y_true != 0
    if not np.any(mask):
        return np.nan
    y_true = y_true[mask]
    y_pred = y_pred[mask]
    denom = np.abs(y_true) + np.abs(y_pred) + eps
    return float(np.mean(2.0 * np.abs(y_pred - y_true) / denom) * 100.0)

def evaluate_on_test00(trained_models, test_00_path):
    # 1) TEST_00 로드 & 정답 준비
    test_df = pd.read_csv(test_00_path)
    test_df = add_features(test_df)
    test_df.loc[test_df['매출수량'] < 0, '매출수량'] = 0
    keys_in_test = set(test_df['영업장명_메뉴명'].astype(str).unique())

    # key별 ground truth
    y_true_map = {}
    for key, g in test_df.groupby('영업장명_메뉴명'):
        g = g.sort_values('영업일자')
        y_true_map[str(key)] = g['매출수량'].to_numpy()

    # 2) 예측
    y_pred_map = predict_next_28_from_train(trained_models, keys=keys_in_test)

    # 3) SMAPE 계산
    rows, all_true, all_pred = [], [], []
    for key in sorted(keys_in_test):
        if key not in y_pred_map or key not in y_true_map:
            continue
        y_pred = y_pred_map[key]
        y_true = y_true_map[key]
        m = min(len(y_true), len(y_pred))
        y_true = y_true[:m]
        y_pred = y_pred[:m]
        s = smape_np(y_true, y_pred)
        rows.append({'영업장명_메뉴명': key, 'n_val': m, 'SMAPE(%)': s})
        all_true.append(y_true)
        all_pred.append(y_pred)

    per_series_df = pd.DataFrame(rows).sort_values('SMAPE(%)')
    overall = smape_np(np.concatenate(all_true), np.concatenate(all_pred)) if rows else np.nan
    return overall, per_series_df, y_pred_map

## 2branch 1Dcnn + bilstm (얼리스타핑 + 기본 변수)

### Train

In [ ]:
# 데이터셋 클래스
class TimeSeriesDataset(Dataset):
    def __init__(self, sequences, targets):
        self.X = sequences
        self.y = targets

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

class DualBranchCNNBiLSTM(nn.Module):
    def __init__(self,
                 cnn_in: int,
                 exog_dim: int,
                 cnn_feat_idx: list,
                 exog_feat_idx: list,
                 hidden_dim: int = 64,
                 num_layers: int = 2,
                 cnn_channels: int = 64,
                 output_dim: int = 7,
                 dropout: float = 0.3):
        super().__init__()
        assert cnn_in >= 1, "CNN 입력 채널(cnn_in)은 최소 1이어야 합니다."

        self.cnn_feat_idx = cnn_feat_idx
        self.exog_feat_idx = exog_feat_idx

        # --- Temporal CNN (Separable: depthwise -> pointwise) ---
        self.depthwise = nn.Conv1d(in_channels=cnn_in,
                                   out_channels=cnn_in,
                                   kernel_size=3, padding=1,
                                   groups=cnn_in)  # 각 채널 독립 필터
        self.pointwise = nn.Conv1d(in_channels=cnn_in,
                                   out_channels=cnn_channels,
                                   kernel_size=1)
        self.cnn_act = nn.ReLU()
        self.cnn_drop = nn.Dropout(dropout)

        # --- BiLSTM on top of CNN feature maps ---
        self.temporal_lstm = nn.LSTM(
            input_size=cnn_channels,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout,
            bidirectional=True
        )

        # --- Exogenous last-step MLP ---
        self.exog_dim = exog_dim
        if self.exog_dim > 0:
            self.exog_mlp = nn.Sequential(
                nn.Linear(self.exog_dim, 64),
                nn.ReLU(),
                nn.Dropout(dropout),
                nn.Linear(64, 64),
                nn.ReLU()
            )
            exog_out_dim = 64
        else:
            self.exog_mlp = None
            exog_out_dim = 0

        # --- Head ---
        self.head = nn.Sequential(
            nn.Dropout(dropout),
            nn.Linear(hidden_dim*2 + exog_out_dim, output_dim)
        )

    def forward(self, x):  # x: (B, T, F)
        # 선택된 CNN 입력 채널만 추출
        x_cnn = x[:, :, self.cnn_feat_idx]           # (B, T, C_cnn)
        s = x_cnn.transpose(1, 2)                    # (B, C_cnn, T)
        s = self.depthwise(s)                        # (B, C_cnn, T)
        s = self.pointwise(s)                        # (B, C_out, T)
        s = self.cnn_act(s)
        s = self.cnn_drop(s)
        s = s.transpose(1, 2)                        # (B, T, C_out)
        s, _ = self.temporal_lstm(s)                 # (B, T, 2H)
        s_last = s[:, -1, :]                         # (B, 2H)

        # 마지막 시점 Exog
        if self.exog_dim > 0 and self.exog_mlp is not None and len(self.exog_feat_idx) > 0:
            ex_last = x[:, -1, self.exog_feat_idx]   # (B, F_exog)
            ex_emb  = self.exog_mlp(ex_last)         # (B, 64)
            feat = torch.cat([s_last, ex_emb], dim=1)
        else:
            feat = s_last

        out = self.head(feat)                        # (B, output_dim)
        return out

In [ ]:
# -----------------------------
# 피처 분할 헬퍼
# - feature_cols를 받아 CNN용/Exog용 컬럼명 리스트로 분할
# -----------------------------
SHORT_WIN_MAX = 14  # 단기 창 기준 (필요시 조정)

_lag_pat = re.compile(r".*?(?:^|_)lag[_-]?(\d+)$", re.IGNORECASE)
_roll_mean_pat = re.compile(r".*?(?:^|_)(?:rmean|ma|ema)[_-]?(\d+)$", re.IGNORECASE)
_roll_std_pat  = re.compile(r".*?(?:^|_)rstd[_-]?(\d+)$", re.IGNORECASE)
_roll_min_pat  = re.compile(r".*?(?:^|_)rmin[_-]?(\d+)$", re.IGNORECASE)
_roll_max_pat  = re.compile(r".*?(?:^|_)rmax[_-]?(\d+)$", re.IGNORECASE)
_diff_pat      = re.compile(r".*?(?:^|_)(?:diff|rdiff)[_-]?(\d+)$", re.IGNORECASE)

def _parse_win(name: str):
    """컬럼명에서 창 길이(숫자) 추출. 없으면 None"""
    for pat in (_lag_pat, _roll_mean_pat, _roll_std_pat, _roll_min_pat, _roll_max_pat, _diff_pat):
        m = pat.match(name)
        if m:
            try:
                return int(m.group(1))
            except:
                return None
    return None

def split_cnn_exog_features(feature_cols):
    """
    규칙
      - '매출수량'은 항상 CNN
      - lag / (rmean, ma, ema) / (rstd, rmin, rmax) / diff, rdiff:
          창 길이 <= SHORT_WIN_MAX → CNN, 그 외 → Exog
      - 나머지(가격/프로모션/캘린더/날씨/카테고리 등) → Exog
    """
    cnn_cols, exog_cols = [], []
    for c in feature_cols:
        if c == '매출수량':
            cnn_cols.append(c); continue
        win = _parse_win(c)
        if win is not None:
            if win <= SHORT_WIN_MAX:
                cnn_cols.append(c)
            else:
                exog_cols.append(c)
        else:
            exog_cols.append(c)
    # 순서 보존 보장
    return cnn_cols, exog_cols

In [ ]:
def train_lstm_with_features(train_df, patience=10):
    trained_models = {}

    train_df = add_features(train_df)
    negative_sales = train_df[train_df['매출수량'] < 0]
    print(negative_sales)
    train_df.loc[train_df['매출수량'] < 0, '매출수량'] = 0

    for store_menu, group in tqdm(train_df.groupby(['영업장명_메뉴명']), desc='Training CNN+BiLSTM'):
        group = group.sort_values('영업일자')
        if len(group) < LOOKBACK + PREDICT:
            continue

        if isinstance(store_menu, tuple):
            store_menu = store_menu[0]

        # 메뉴별 feature 컬럼 선택
        feature_cols = ['매출수량'] + menu_rfecv_features.get(store_menu, [])

        # --- 라우팅: CNN용 / Exog용 ---
        cnn_cols, exog_cols = split_cnn_exog_features(feature_cols)
        if '매출수량' not in cnn_cols:
            cnn_cols = ['매출수량'] + [c for c in cnn_cols if c != '매출수량']

        # 스케일링
        scaler = MinMaxScaler()
        group[feature_cols] = scaler.fit_transform(group[feature_cols])

        # 윈도우 생성
        sequences, targets = [], []
        values = group[feature_cols].values
        for i in range(len(values) - LOOKBACK - PREDICT + 1):
            sequences.append(values[i:i + LOOKBACK])
            targets.append(values[i + LOOKBACK:i + LOOKBACK + PREDICT, 0])

        if not sequences:
            continue

        sequences = torch.from_numpy(np.array(sequences, dtype=np.float32))
        targets   = torch.from_numpy(np.array(targets, dtype=np.float32))

        dataset = TimeSeriesDataset(sequences, targets)
        loader = DataLoader(dataset, batch_size=32, shuffle=True)

        # 인덱스 매핑
        cnn_feat_idx  = [feature_cols.index(c) for c in cnn_cols]
        exog_feat_idx = [feature_cols.index(c) for c in exog_cols] if exog_cols else []

        # 하이퍼파라미터
        local_epochs = 119
        lr = 0.0014389257980049275
        grad_clip = 0.5

        model = DualBranchCNNBiLSTM(
            cnn_in=len(cnn_feat_idx),
            exog_dim=len(exog_feat_idx),
            cnn_feat_idx=cnn_feat_idx,
            exog_feat_idx=exog_feat_idx,
            hidden_dim=48,
            num_layers=2,
            cnn_channels=64,
            output_dim=PREDICT,
            dropout=0.3502700784447087
        ).to(DEVICE)

        optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=0.0023983603243525192)
        steps_per_epoch = max(1, len(loader))
        scheduler = torch.optim.lr_scheduler.OneCycleLR(
            optimizer,
            max_lr=lr * 2.0,
            epochs=local_epochs,
            steps_per_epoch=steps_per_epoch,
            pct_start=0.3
        )
        criterion = smape_loss

        # === Early Stopping 준비 ===
        best_loss = float('inf')
        wait = 0
        best_model_state = None

        # 학습 루프
        model.train()
        for epoch in range(local_epochs):
            total_loss = 0.0
            for x_batch, y_batch in loader:
                x_batch, y_batch = x_batch.to(DEVICE), y_batch.to(DEVICE)
                output = model(x_batch)
                loss = criterion(output, y_batch)
                optimizer.zero_grad()
                loss.backward()
                if grad_clip and grad_clip > 0:
                    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=grad_clip)
                optimizer.step()
                scheduler.step()
                total_loss += loss.item()

            epoch_loss = total_loss / len(loader)
            # print(f"[{store_menu}] Epoch {epoch+1}/{local_epochs}, Loss={epoch_loss:.4f}")

            # === Early Stopping check ===
            if epoch_loss < best_loss:
                best_loss = epoch_loss
                wait = 0
                best_model_state = model.state_dict()
            else:
                wait += 1
            if wait >= patience:
                print(f"⏹️ {store_menu}: Early stopping at epoch {epoch+1}, best_loss={best_loss:.4f}")
                if best_model_state is not None:
                    model.load_state_dict(best_model_state)
                break

        trained_models[store_menu] = {
            'model': model.eval(),
            'scaler': scaler,
            'feature_cols': feature_cols,
            'cnn_feature_cols': cnn_cols,
            'exog_feature_cols': exog_cols,
            'cnn_feat_idx': cnn_feat_idx,
            'exog_feat_idx': exog_feat_idx,
            'last_sequence': values[-LOOKBACK:]
        }

    return trained_models


In [ ]:
# 학습 실행
trained_models = train_lstm_with_features(train_df=train)

            영업일자                영업장명_메뉴명  매출수량  요일   월   일  분기  주말여부  공휴일여부  \
1837  2023-08-30  느티나무 셀프BBQ_대여료 60,000원    -1   2   8  30   3     0      0   
3453  2023-09-19   느티나무 셀프BBQ_스프라이트 (단체)   -18   1   9  19   3     0      0   
55068 2023-09-30        미라시아_브런치 4인 패키지     -1   5   9  30   3     1      1   
56134 2023-10-02         미라시아_브런치(대인) 주중    -5   0  10   2   4     0      1   
64507 2023-05-16           연회장_Cass Beer   -18   1   5  16   2     0      0   
64849 2024-04-22           연회장_Cass Beer   -26   0   4  22   2     0      0   
65585 2023-05-30       연회장_Conference L2    -1   1   5  30   2     0      0   
65693 2023-09-15       연회장_Conference L2    -1   4   9  15   3     0      0   
78401 2023-07-17      카페테리아_단체식 18000(신)   -80   0   7  17   3     0      0   
89446 2023-03-12              포레스트릿_꼬치어묵    -3   6   3  12   1     1      0   
89978 2023-03-12               포레스트릿_떡볶이    -3   6   3  12   1     1      0   
92638 2023-03-12        포레스트릿_아메리카노(ICE)    -1   6  

Training CNN+BiLSTM:   1%|          | 1/193 [00:17<54:29, 17.03s/it]

⏹️ 느티나무 셀프BBQ_1인 수저세트: Early stopping at epoch 94, best_loss=1.1673


Training CNN+BiLSTM:   1%|          | 2/193 [00:22<32:23, 10.18s/it]

⏹️ 느티나무 셀프BBQ_BBQ55(단체): Early stopping at epoch 51, best_loss=1.5094


...

⏹️ 화담숲카페_아메리카노 ICE: Early stopping at epoch 83, best_loss=1.1327


Training CNN+BiLSTM:  99%|█████████▉| 192/193 [25:42<00:07,  7.93s/it]

⏹️ 화담숲카페_카페라떼 ICE: Early stopping at epoch 71, best_loss=1.1858


Training CNN+BiLSTM: 100%|██████████| 193/193 [25:53<00:00,  8.05s/it]

⏹️ 화담숲카페_현미뻥스크림: Early stopping at epoch 112, best_loss=1.1654


### Validation

In [ ]:
# 한달 검증 코드
overall, per_series_df, per_window_df, long_df = rolling_validate_real_dates(
    trained_models,
    train_df=train,          # 2024-06-15까지
    test_df=pd.read_csv('/content/drive/MyDrive/dacon/open/test/TEST_00.csv')  # 이미 실날짜 형식
)
print(f"[Overall SMAPE] {overall:.3f}%")

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/valida

[Overall SMAPE] 54.823%


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


In [ ]:
menu_smape_df = menu_smape_table(long_df)
menu_smape_df

,영업장명_메뉴명,n_days,SMAPE(%)
50,담하_제로콜라,126,0.540385
121,연회장_Conference M1,154,0.726695
100,미라시아_브런치 4인 패키지,112,0.979509
127,연회장_OPUS 2,133,1.123288
54,담하_콜라,147,3.857420
...,...,...,...
40,담하_라면사리,154,106.499504
163,포레스트릿_코카콜라,154,168.505961
148,카페테리아_오픈푸드,84,175.668697
157,포레스트릿_스프라이트,147,195.107304


In [ ]:
# TEST_00로 검증 예측
def predict_next_28_from_train(trained_models, keys=None):
    preds = {}
    for key, pack in trained_models.items():
        key_str = key[0] if isinstance(key, tuple) else key  # tuple → 문자열 변환
        if (keys is not None) and (key_str not in keys):
            continue
        model = pack['model']
        scaler = pack['scaler']
        feature_cols = pack['feature_cols']
        recent_scaled = pack['last_sequence']  # 이미 scaled

        x_input = torch.tensor([recent_scaled]).float().to(DEVICE)
        with torch.no_grad():
            pred_scaled = model(x_input).squeeze().cpu().numpy()

        # 역스케일 (매출수량만 역변환)
        restored = []
        for v in pred_scaled:
            dummy = np.zeros((1, len(feature_cols)))
            dummy[0, 0] = v
            inv = scaler.inverse_transform(dummy)[0, 0]
            restored.append(max(inv, 0.0))
        preds[key_str] = np.array(restored, dtype=float)
    return preds

# TEST_00 정답과 SMAPE 평가
def smape_np(y_true, y_pred, eps: float = 1e-6):
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    mask = y_true != 0
    if not np.any(mask):
        return np.nan
    y_true = y_true[mask]
    y_pred = y_pred[mask]
    denom = np.abs(y_true) + np.abs(y_pred) + eps
    return float(np.mean(2.0 * np.abs(y_pred - y_true) / denom) * 100.0)

def evaluate_on_test00(trained_models, test_00_path):
    # 1) TEST_00 로드 & 정답 준비
    test_df = pd.read_csv(test_00_path)
    test_df = add_features(test_df)
    test_df.loc[test_df['매출수량'] < 0, '매출수량'] = 0
    keys_in_test = set(test_df['영업장명_메뉴명'].astype(str).unique())

    # key별 ground truth
    y_true_map = {}
    for key, g in test_df.groupby('영업장명_메뉴명'):
        g = g.sort_values('영업일자')
        y_true_map[str(key)] = g['매출수량'].to_numpy()

    # 2) 예측
    y_pred_map = predict_next_28_from_train(trained_models, keys=keys_in_test)

    # 3) SMAPE 계산
    rows, all_true, all_pred = [], [], []
    for key in sorted(keys_in_test):
        if key not in y_pred_map or key not in y_true_map:
            continue
        y_pred = y_pred_map[key]
        y_true = y_true_map[key]
        m = min(len(y_true), len(y_pred))
        y_true = y_true[:m]
        y_pred = y_pred[:m]
        s = smape_np(y_true, y_pred)
        rows.append({'영업장명_메뉴명': key, 'n_val': m, 'SMAPE(%)': s})
        all_true.append(y_true)
        all_pred.append(y_pred)

    per_series_df = pd.DataFrame(rows).sort_values('SMAPE(%)')
    overall = smape_np(np.concatenate(all_true), np.concatenate(all_pred)) if rows else np.nan
    return overall, per_series_df, y_pred_map

In [ ]:
# 일주일 검증 코드
overall_smape, per_series_smape_df, preds_map = evaluate_on_test00(
     trained_models,
     '/content/drive/MyDrive/dacon/open/test/TEST_00.csv'
 )
print(f'[Overall SMAPE] {overall_smape:.3f}%')

/tmp/ipython-input-455680956.py:13: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:253.)
  x_input = torch.tensor([recent_scaled]).float().to(DEVICE)


[Overall SMAPE] 50.448%


In [ ]:
per_series_smape_df

,영업장명_메뉴명,n_val,SMAPE(%)
174,포레스트릿_아메리카노(ICE),7,0.064389
130,연회장_Grand Ballroom,7,0.157604
123,연회장_Conference L2,7,0.264843
53,담하_제로콜라,7,0.479447
57,담하_콜라,7,0.736732
119,미라시아_파스타면 추가(150g),7,0.859733
67,라그로타_Gls.Sileni,7,0.940282
131,연회장_OPUS 2,7,1.041888
127,연회장_Conference M9,7,1.193333
114,미라시아_잭 애플 토닉,7,1.216568


### Predict

In [ ]:
def predict_with_trained_models(test_df, trained_models, test_prefix):
    """
    test_df: 예측할 원본 데이터프레임
    trained_models: train_lstm_with_features()에서 학습된 모델 dict
    test_prefix: 'TEST_00' 같은 접두어
    """
    # --- 피처 생성 & 전처리 ---
    test_df = add_features(test_df.copy())
    negative_sales = test_df[test_df['매출수량'] < 0]
    print(f"{test_prefix} 음수 매출수량 행 개수: {negative_sales.shape[0]}")
    if not negative_sales.empty:
        print(negative_sales)
    test_df.loc[test_df['매출수량'] < 0, '매출수량'] = 0

    results = []

    # --- 메뉴별 예측 ---
    for store_menu, group in test_df.groupby(['영업장명_메뉴명']):
        key = store_menu if not isinstance(store_menu, tuple) else store_menu[0]
        if key not in trained_models:
            continue

        pack = trained_models[key]
        model        = pack['model']
        scaler       = pack['scaler']
        feature_cols = pack['feature_cols']

        group = group.sort_values('영업일자')

        # 최근 LOOKBACK 확보
        if len(group) < LOOKBACK:
            continue
        recent_df = group.iloc[-LOOKBACK:][feature_cols]
        recent_scaled = scaler.transform(recent_df)

        # 모델 입력
        x_input = torch.tensor([recent_scaled], dtype=torch.float32).to(DEVICE)
        with torch.no_grad():
            pred_scaled = model(x_input).squeeze(0).cpu().numpy()  # (PREDICT,)

        # 역스케일링 (매출수량만)
        restored = []
        for v in pred_scaled:
            dummy = np.zeros((1, len(feature_cols)), dtype=float)
            dummy[0, 0] = v
            inv = scaler.inverse_transform(dummy)[0, 0]
            restored.append(max(inv, 0.0))

        # 결과 저장
        pred_dates = [f"{test_prefix}+{i+1}일" for i in range(PREDICT)]
        for d, val in zip(pred_dates, restored):
            results.append({
                '영업일자': d,
                '영업장명_메뉴명': key,
                '매출수량': val
            })

    return pd.DataFrame(results)

In [ ]:
all_preds = []
test_files = sorted(glob.glob('/content/drive/MyDrive/dacon/open/test/TEST_*.csv'))

for path in test_files:
    test_df = pd.read_csv(path)
    filename = os.path.basename(path)
    test_prefix = re.search(r'(TEST_\d+)', filename).group(1)

    pred_df = predict_with_trained_models(test_df, trained_models, test_prefix)
    all_preds.append(pred_df)

full_pred_df = pd.concat(all_preds, ignore_index=True)


TEST_00 음수 매출수량 행 개수: 0
TEST_01 음수 매출수량 행 개수: 0
TEST_02 음수 매출수량 행 개수: 4
           영업일자                    영업장명_메뉴명  매출수량  요일  월   일  분기  주말여부  공휴일여부  \
407  2024-09-09  느티나무 셀프BBQ_잔디그늘집 대여료 (6인석)    -1   0  9   9   3     0      0   
828  2024-09-10                담하_(정식) 물냉면     -1   1  9  10   3     0      0   
1518 2024-08-31                      담하_참이슬    -6   5  8  31   3     1      0   
1574 2024-08-31                       담하_카스    -9   5  8  31   3     1      0   

      휴일전날  ...  27일전매출    요일별_평균  요일별_중앙값  요일별_표준편차      월별_평균  월별_중앙값  \
407      0  ...     NaN  2.815789      0.0  8.377288  18.900000    14.5   
828      0  ...     NaN  2.394737      0.0  3.780931   3.633333     1.5   
1518     0  ...     NaN  4.828947      5.0  3.515255   2.548387     2.0   
1574     0  ...     NaN  2.526316      2.0  2.532317   1.387097     1.0   

        월별_표준편차  월요일_평균   요일평균_대비    월평균_대비  
407   17.889218    0.00 -1.355140 -1.052910  
828    4.916428    0.25 -1.417582 -1.275229  
1518   2

In [ ]:
full_pred_df

,영업일자,영업장명_메뉴명,매출수량
0,TEST_00+1일,느티나무 셀프BBQ_1인 수저세트,4.962111
1,TEST_00+2일,느티나무 셀프BBQ_1인 수저세트,5.746692
2,TEST_00+3일,느티나무 셀프BBQ_1인 수저세트,4.524084
3,TEST_00+4일,느티나무 셀프BBQ_1인 수저세트,5.305899
4,TEST_00+5일,느티나무 셀프BBQ_1인 수저세트,4.981863
...,...,...,...
13505,TEST_09+3일,화담숲카페_현미뻥스크림,48.432003
13506,TEST_09+4일,화담숲카페_현미뻥스크림,31.043636
13507,TEST_09+5일,화담숲카페_현미뻥스크림,25.735215
13508,TEST_09+6일,화담숲카페_현미뻥스크림,43.378057


## Submission

In [ ]:
# 제출 파일 포맷 변환 함수
def convert_to_submission_format(pred_df: pd.DataFrame, sample_submission: pd.DataFrame):
    # tuple → string 변환
    pred_df['영업장명_메뉴명'] = pred_df['영업장명_메뉴명'].apply(
        lambda x: x[0] if isinstance(x, tuple) else x
    ).astype(str)

    pred_dict = dict(zip(
        zip(pred_df['영업일자'], pred_df['영업장명_메뉴명']),
        pred_df['매출수량']
    ))

    final_df = sample_submission.copy()
    for row_idx in final_df.index:
        date = final_df.loc[row_idx, '영업일자']
        for col in final_df.columns[1:]:
            final_df.loc[row_idx, col] = pred_dict.get((date, col), 0)
    return final_df

In [ ]:
sample_submission = pd.read_csv('/content/drive/MyDrive/dacon/open/sample_submission.csv')
submission = convert_to_submission_format(full_pred_df, sample_submission)

/tmp/ipython-input-280057456.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '4.962111055850983' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  final_df.loc[row_idx, col] = pred_dict.get((date, col), 0)
/tmp/ipython-input-280057456.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '65.82955335080624' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  final_df.loc[row_idx, col] = pred_dict.get((date, col), 0)
/tmp/ipython-input-280057456.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '5.69345386326313' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  final_df.loc[row_idx, col] = pred_dict.get((date, col), 0)
/tmp/

In [ ]:
# 음수인 경우, 0 / 소수인 경우 정수로 내림
negative_count = 0
for col in submission.columns[1:]:
    negative_count += (submission[col] < 0).sum()
    submission[col] = submission[col].apply(lambda x: int(np.floor(max(x, 0))))

print(f"음수였던 값 개수: {negative_count}")
print(submission)
submission



음수였던 값 개수: 0
          영업일자  느티나무 셀프BBQ_1인 수저세트  느티나무 셀프BBQ_BBQ55(단체)  \
0   TEST_00+1일                   4                    65   
1   TEST_00+2일                   5                    53   
2   TEST_00+3일                   4                    56   
3   TEST_00+4일                   5                    65   
4   TEST_00+5일                   4                    64   
..         ...                 ...                   ...   
65  TEST_09+3일                   3                    73   
66  TEST_09+4일                   4                    74   
67  TEST_09+5일                   4                    79   
68  TEST_09+6일                   7                    79   
69  TEST_09+7일                   6                    83   

    느티나무 셀프BBQ_대여료 30,000원  느티나무 셀프BBQ_대여료 60,000원  느티나무 셀프BBQ_대여료 90,000원  \
0                        5                       1                       1   
1                        4                       1                       1   
2                        1      

,영업일자,느티나무 셀프BBQ_1인 수저세트,느티나무 셀프BBQ_BBQ55(단체),"느티나무 셀프BBQ_대여료 30,000원","느티나무 셀프BBQ_대여료 60,000원","느티나무 셀프BBQ_대여료 90,000원","느티나무 셀프BBQ_본삼겹 (단품,실내)",느티나무 셀프BBQ_스프라이트 (단체),느티나무 셀프BBQ_신라면,느티나무 셀프BBQ_쌈야채세트,...,화담숲주막_스프라이트,화담숲주막_참살이 막걸리,화담숲주막_찹쌀식혜,화담숲주막_콜라,화담숲주막_해물파전,화담숲카페_메밀미숫가루,화담숲카페_아메리카노 HOT,화담숲카페_아메리카노 ICE,화담숲카페_카페라떼 ICE,화담숲카페_현미뻥스크림
0,TEST_00+1일,4,65,5,1,1,1,21,4,3,...,14,21,13,12,45,49,4,39,5,25
1,TEST_00+2일,5,53,4,1,1,1,14,4,1,...,7,21,14,7,22,29,4,29,4,14
2,TEST_00+3일,4,56,1,1,0,1,12,4,1,...,6,15,9,5,20,19,4,26,3,11
3,TEST_00+4일,5,65,2,1,1,1,17,2,1,...,4,12,5,5,20,16,3,17,4,7
4,TEST_00+5일,4,64,2,1,1,1,20,1,3,...,3,10,4,4,23,20,4,27,5,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,TEST_09+3일,3,73,1,1,1,2,13,3,2,...,9,43,16,9,44,66,39,83,16,48
66,TEST_09+4일,4,74,1,1,1,1,17,3,3,...,5,29,11,10,47,58,33,74,9,31
67,TEST_09+5일,4,79,5,1,1,2,22,7,4,...,5,27,12,6,63,61,42,83,9,25
68,TEST_09+6일,7,79,11,2,2,3,30,7,8,...,10,39,22,13,137,73,36,92,13,43


# 0을 메뉴별 월 별 최빈값으로 대체하기

In [ ]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

# --- 0. 기본 설정 및 데이터 로드 (이전과 동일) ---
TEST_CSV_PATH = '/content/drive/MyDrive/dacon/open/test'
pred_df = submission # submission 변수를 사용
train_df = pd.read_csv("/content/drive/MyDrive/dacon/open/train/train.csv")

# ----------------------------------------

# --- 1. 날짜 정보 생성 (이전과 동일) ---
print("Step 1: 예측 데이터셋의 실제 날짜 및 월 정보를 생성합니다...")
pred_df = pred_df.set_index('영업일자')
actual_dates = []
for i in range(10):
    test_file_path = os.path.join(TEST_CSV_PATH, f'TEST_{i:02d}.csv')
    if os.path.exists(test_file_path):
        temp_df = pd.read_csv(test_file_path)
        last_date = pd.to_datetime(temp_df['영업일자']).max()
        forecast_dates = pd.date_range(start=last_date + pd.Timedelta(days=1), periods=7)
        actual_dates.extend(forecast_dates)
    else:
        raise FileNotFoundError(f"{test_file_path} not found. 경로를 확인해주세요.")

if len(actual_dates) == len(pred_df):
    pred_df['월'] = pd.Series(actual_dates, index=pred_df.index).dt.month
    print("'월' 컬럼이 임시로 추가되었습니다.")
else:
    raise ValueError("생성된 날짜 수와 예측 데이터 행 수가 일치하지 않습니다.")

# ----------------------------------------

# --- 2. 최빈값 조견표 생성 (이전과 동일) ---
print("\nStep 2: Train 데이터에서 월별/메뉴별 판매수량 최빈값을 계산합니다...")
train_df['영업일자'] = pd.to_datetime(train_df['영업일자'])
train_df['월'] = train_df['영업일자'].dt.month
train_sales_df = train_df[train_df['매출수량'] > 0]
monthly_modes = train_sales_df.groupby(['월', '영업장명_메뉴명'])['매출수량'].apply(
    lambda x: x.mode().iloc[0] if not x.mode().empty else None
)
mode_map = monthly_modes.to_dict()
print("최빈값 조견표(mode_map) 생성이 완료되었습니다.")

# ----------------------------------------

# --- 3. 0인 값만 찾아서 대체하기 (확인 코드 추가) ---
print("\nStep 3: 예측 데이터의 0 값만 최빈값으로 대체합니다...")

menu_columns = [col for col in pred_df.columns if col != '월']
pred_df_imputed = pred_df.copy()

# ### --- 추가된 부분: 미대체 0을 기록할 리스트 --- ###
unreplaced_zeros = []
# ### ----------------------------------------- ###

for menu_col in tqdm(menu_columns, desc="메뉴별 0 값 대체 중"):
    for idx in pred_df_imputed.index:
        if pred_df_imputed.loc[idx, menu_col] == 0:
            month = pred_df_imputed.loc[idx, '월']
            imputation_value = mode_map.get((month, menu_col), 0)

            if imputation_value > 0:
                pred_df_imputed.loc[idx, menu_col] = imputation_value
            else:
                # ### --- 추가된 부분: 왜 0이 남았는지 기록 --- ###
                # 최빈값을 찾지 못했거나(None), 최빈값이 0인 경우
                unreplaced_zeros.append({'월': month, '메뉴명': menu_col, '이유': 'Train 데이터에 해당 월의 판매 기록 없음'})
                # ### ----------------------------------------- ###

print("0 값 대체 작업이 완료되었습니다.")

# ----------------------------------------

# --- 3.5. 미대체 0값 확인 및 원인 분석 ---
print("\nStep 3.5: 대체되지 않고 0으로 남아있는 값들을 확인합니다...")

remaining_zeros_count = (pred_df_imputed[menu_columns] == 0).sum().sum()
print(f"총 {remaining_zeros_count}개의 0이 여전히 남아있습니다.")

if unreplaced_zeros:
    unreplaced_df = pd.DataFrame(unreplaced_zeros).drop_duplicates().sort_values(by=['월', '메뉴명'])
    print("\n[원인 분석] 아래 월/메뉴 조합은 Train 데이터에 판매 기록이 없어 최빈값을 찾지 못했습니다:")

    pd.set_option('display.max_rows', None)
    display(unreplaced_df)
    pd.reset_option('display.max_rows')

# ----------------------------------------

# --- 4. 최종 결과물 정리 및 확인 (이전과 동일) ---
print("\nStep 4: 최종 제출 형식에 맞게 불필요한 컬럼을 제거합니다...")
final_pred_df = pred_df_imputed.drop(columns=['월'])
final_pred_df = final_pred_df.reset_index()

print("\n[대체 후] 최종 결과 확인 (상위 5개 행):")
display(final_pred_df.head())

Step 1: 예측 데이터셋의 실제 날짜 및 월 정보를 생성합니다...
'월' 컬럼이 임시로 추가되었습니다.

Step 2: Train 데이터에서 월별/메뉴별 판매수량 최빈값을 계산합니다...
최빈값 조견표(mode_map) 생성이 완료되었습니다.

Step 3: 예측 데이터의 0 값만 최빈값으로 대체합니다...


메뉴별 0 값 대체 중: 100%|██████████| 193/193 [00:00<00:00, 369.18it/s]

0 값 대체 작업이 완료되었습니다.

Step 3.5: 대체되지 않고 0으로 남아있는 값들을 확인합니다...
총 71개의 0이 여전히 남아있습니다.

[원인 분석] 아래 월/메뉴 조합은 Train 데이터에 판매 기록이 없어 최빈값을 찾지 못했습니다:


,월,메뉴명,이유
0,1,담하_갑오징어 비빔밥,Train 데이터에 해당 월의 판매 기록 없음
1,2,담하_갑오징어 비빔밥,Train 데이터에 해당 월의 판매 기록 없음
31,4,카페테리아_샷 추가,Train 데이터에 해당 월의 판매 기록 없음
43,4,카페테리아_아메리카노(HOT),Train 데이터에 해당 월의 판매 기록 없음
17,5,카페테리아_구슬아이스크림,Train 데이터에 해당 월의 판매 기록 없음
33,5,카페테리아_샷 추가,Train 데이터에 해당 월의 판매 기록 없음
49,5,카페테리아_아메리카노(HOT),Train 데이터에 해당 월의 판매 기록 없음
56,5,카페테리아_오픈푸드,Train 데이터에 해당 월의 판매 기록 없음
2,7,라그로타_모둠 해산물 플래터,Train 데이터에 해당 월의 판매 기록 없음
10,7,미라시아_잭 애플 토닉,Train 데이터에 해당 월의 판매 기록 없음



Step 4: 최종 제출 형식에 맞게 불필요한 컬럼을 제거합니다...

[대체 후] 최종 결과 확인 (상위 5개 행):


,영업일자,느티나무 셀프BBQ_1인 수저세트,느티나무 셀프BBQ_BBQ55(단체),"느티나무 셀프BBQ_대여료 30,000원","느티나무 셀프BBQ_대여료 60,000원","느티나무 셀프BBQ_대여료 90,000원","느티나무 셀프BBQ_본삼겹 (단품,실내)",느티나무 셀프BBQ_스프라이트 (단체),느티나무 셀프BBQ_신라면,느티나무 셀프BBQ_쌈야채세트,...,화담숲주막_스프라이트,화담숲주막_참살이 막걸리,화담숲주막_찹쌀식혜,화담숲주막_콜라,화담숲주막_해물파전,화담숲카페_메밀미숫가루,화담숲카페_아메리카노 HOT,화담숲카페_아메리카노 ICE,화담숲카페_카페라떼 ICE,화담숲카페_현미뻥스크림
0,TEST_00+1일,4,65,5,1,1,1,21,4,3,...,14,21,13,12,45,49,4,39,5,25
1,TEST_00+2일,5,53,4,1,1,1,14,4,1,...,7,21,14,7,22,29,4,29,4,14
2,TEST_00+3일,4,56,1,1,1,1,12,4,1,...,6,15,9,5,20,19,4,26,3,11
3,TEST_00+4일,5,65,2,1,1,1,17,2,1,...,4,12,5,5,20,16,3,17,4,7
4,TEST_00+5일,4,64,2,1,1,1,20,1,3,...,3,10,4,4,23,20,4,27,5,6


In [ ]:
final_pred_df
# 여전히 0이 남아있는 이유? 그 메뉴가 그 달에 한번도 안팔려서

,영업일자,느티나무 셀프BBQ_1인 수저세트,느티나무 셀프BBQ_BBQ55(단체),"느티나무 셀프BBQ_대여료 30,000원","느티나무 셀프BBQ_대여료 60,000원","느티나무 셀프BBQ_대여료 90,000원","느티나무 셀프BBQ_본삼겹 (단품,실내)",느티나무 셀프BBQ_스프라이트 (단체),느티나무 셀프BBQ_신라면,느티나무 셀프BBQ_쌈야채세트,...,화담숲주막_스프라이트,화담숲주막_참살이 막걸리,화담숲주막_찹쌀식혜,화담숲주막_콜라,화담숲주막_해물파전,화담숲카페_메밀미숫가루,화담숲카페_아메리카노 HOT,화담숲카페_아메리카노 ICE,화담숲카페_카페라떼 ICE,화담숲카페_현미뻥스크림
0,TEST_00+1일,4,65,5,1,1,1,21,4,3,...,14,21,13,12,45,49,4,39,5,25
1,TEST_00+2일,5,53,4,1,1,1,14,4,1,...,7,21,14,7,22,29,4,29,4,14
2,TEST_00+3일,4,56,1,1,1,1,12,4,1,...,6,15,9,5,20,19,4,26,3,11
3,TEST_00+4일,5,65,2,1,1,1,17,2,1,...,4,12,5,5,20,16,3,17,4,7
4,TEST_00+5일,4,64,2,1,1,1,20,1,3,...,3,10,4,4,23,20,4,27,5,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,TEST_09+3일,3,73,1,1,1,2,13,3,2,...,9,43,16,9,44,66,39,83,16,48
66,TEST_09+4일,4,74,1,1,1,1,17,3,3,...,5,29,11,10,47,58,33,74,9,31
67,TEST_09+5일,4,79,5,1,1,2,22,7,4,...,5,27,12,6,63,61,42,83,9,25
68,TEST_09+6일,7,79,11,2,2,3,30,7,8,...,10,39,22,13,137,73,36,92,13,43


In [ ]:
# 0을 1로 바꿔주기
import pandas as pd

# final_pred_df 변수가 이미 있다고 가정합니다.

print("--- 변경 전 ---")
# 변경 전 0의 개수를 확인 (선택 사항)
numeric_cols = final_pred_df.select_dtypes(include=np.number).columns
print("변경 전 0의 개수:", (final_pred_df[numeric_cols] == 0).sum().sum())

# --- 핵심 코드 ---
# final_pred_df의 모든 0을 1로 변경합니다.
submission = final_pred_df.replace(0, 1)
# ----------------

print("\n--- 변경 후 ---")
# 변경 후 상위 5개 행을 확인하여 0이 1로 바뀌었는지 봅니다.
display(submission.head())

# 변경 후 0이 하나도 없는지 최종 확인 (선택 사항)
numeric_cols = submission.select_dtypes(include=np.number).columns
print("\n변경 후 0의 개수:", (submission[numeric_cols] == 0).sum().sum())

--- 변경 전 ---
변경 전 0의 개수: 71

--- 변경 후 ---


,영업일자,느티나무 셀프BBQ_1인 수저세트,느티나무 셀프BBQ_BBQ55(단체),"느티나무 셀프BBQ_대여료 30,000원","느티나무 셀프BBQ_대여료 60,000원","느티나무 셀프BBQ_대여료 90,000원","느티나무 셀프BBQ_본삼겹 (단품,실내)",느티나무 셀프BBQ_스프라이트 (단체),느티나무 셀프BBQ_신라면,느티나무 셀프BBQ_쌈야채세트,...,화담숲주막_스프라이트,화담숲주막_참살이 막걸리,화담숲주막_찹쌀식혜,화담숲주막_콜라,화담숲주막_해물파전,화담숲카페_메밀미숫가루,화담숲카페_아메리카노 HOT,화담숲카페_아메리카노 ICE,화담숲카페_카페라떼 ICE,화담숲카페_현미뻥스크림
0,TEST_00+1일,4,65,5,1,1,1,21,4,3,...,14,21,13,12,45,49,4,39,5,25
1,TEST_00+2일,5,53,4,1,1,1,14,4,1,...,7,21,14,7,22,29,4,29,4,14
2,TEST_00+3일,4,56,1,1,1,1,12,4,1,...,6,15,9,5,20,19,4,26,3,11
3,TEST_00+4일,5,65,2,1,1,1,17,2,1,...,4,12,5,5,20,16,3,17,4,7
4,TEST_00+5일,4,64,2,1,1,1,20,1,3,...,3,10,4,4,23,20,4,27,5,6



변경 후 0의 개수: 0


In [ ]:
submission.to_csv('/content/drive/MyDrive/dacon/0824_cnn_lstm_rounding_mode.csv', index=False, encoding='utf-8-sig')